In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [0]:
from package import MakeFeatures, make_predict

R_STATE = 17

In [0]:
PATH_ANS = 'csv'

if not os.path.exists(PATH_ANS):
    os.mkdir(PATH_ANS)

In [0]:
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')

In [7]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [0]:
X_train, y_train = train.drop(['dep_delayed_15min'], axis=1), train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [9]:
mf = MakeFeatures(is_weekend=1, 
                  seasons=1, 
                  day_of_month=1, 
                  dep_capital=1, 
                  arr_capital=1, 
                  route=0, 
                  log_dist=1, 
                  uc=1, 
                  time = 1,
                  new_dep_time = 1,
                  hour_imp = 0)
mf.fit(X_train)

X_train_new = mf.fit_transform(X_train)
X_train_new.head()

Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!


,Seasons_cos,Seasons_sin,IsWeekend,Day,DepFromCap,ArrForCap,LogDist,Hour,Minute,AA,AQ,AS,B6,CO,DH,DL,EV,F9,FL,HA,HP,MQ,NW,OH,OO,TZ,UA,US,WN,XE,YV,day,evening,morning,night
0,-5.000000e-01,-0.866025,1,21,0,0,6.595781,19,34,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,-5.000000e-01,0.866025,0,20,0,0,6.726233,15,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,-1.836970e-16,-1.000000,0,2,0,0,6.030685,14,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
3,8.660254e-01,-0.500000,1,25,0,0,6.770789,10,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,5.000000e-01,-0.866025,1,7,0,0,6.047372,18,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [0]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=R_STATE)

In [12]:
xgb_first = XGBClassifier(
                        objective= 'binary:logistic'
                        )
cross_val_score(xgb_first, X=X_train_new.values, y=y_train, cv=skf, scoring="roc_auc", verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   19.6s finished


array([0.71239174, 0.71476971, 0.71396586])

In [0]:
xgb = XGBClassifier(learning_rate=0.1,
                        max_depth=4,
                        min_child_weight=2,
                        n_estimators=1000,
                        gamma=0.1,
                        subsamle = 0.9,
                        colsample_bytree = 0.5,
                        reg_alpha = 1.4,
                        objective= 'binary:logistic'
                        )

In [14]:
cross_val_score(xgb, X=X_train_new.values, y=y_train, cv=skf, scoring="roc_auc", verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.7min finished


array([0.72723519, 0.73409327, 0.73189532])

In [0]:
mf_test = MakeFeatures(is_weekend=1, 
                  seasons=1, 
                  day_of_month=1, 
                  dep_capital=1, 
                  arr_capital=1, 
                  route=0, 
                  log_dist=1, 
                  uc=1, 
                  time = 1,
                  new_dep_time = 1,
                  hour_imp = 0)
mf_test.fit(test)

X_test = mf_test.fit_transform(test)
X_test.head()

In [0]:
X_test.drop(['9E'], axis=1, inplace=True)

X_test['DH'] = 0
X_test['HP'] = 0
X_test['TZ'] = 0

In [0]:
X_train_new.Day = X_train_new.Day.map(int)

In [0]:
xgb.fit(X_train_new, y_train)

In [0]:
X_test.Day = X_test.Day.map(int)
neworder = X_train_new.columns
X_test_new = X_test.reindex(columns=neworder)

In [0]:
predict_kaggle = xgb.predict_proba(X_test_new)[:, 1]

In [0]:
make_predict(predict_kaggle,'xgb')

# Use all dataset

In [15]:
mf = MakeFeatures(is_weekend=1, 
                  seasons=1, 
                  day_of_month=1, 
                  dep_capital=1, 
                  arr_capital=1, 
                  route=1, 
                  log_dist=1, 
                  uc=1, 
                  time = 1,
                  new_dep_time = 1,
                  hour_imp = 1)
mf.fit(X_train)

X_train_new = mf.fit_transform(X_train)
X_train_new.head()

Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Hour_sq-s feature complete!
Route complete!
Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Hour_sq-s feature complete!
Route complete!


,Seasons_cos,Seasons_sin,IsWeekend,Day,DepFromCap,ArrForCap,LogDist,Hour,Minute,Hour_sq,Hour_sq2,AA,AQ,AS,B6,CO,DH,DL,EV,F9,FL,HA,HP,MQ,NW,OH,OO,TZ,UA,US,WN,XE,YV,ABE_Origin,ABI_Origin,ABQ_Origin,ABY_Origin,ACK_Origin,ACT_Origin,ACV_Origin,...,SMF_Dest,SMX_Dest,SNA_Dest,SOP_Dest,SPI_Dest,SPS_Dest,SRQ_Dest,STL_Dest,STT_Dest,STX_Dest,SUN_Dest,SWF_Dest,SYR_Dest,TEX_Dest,TLH_Dest,TOL_Dest,TPA_Dest,TRI_Dest,TTN_Dest,TUL_Dest,TUP_Dest,TUS_Dest,TVC_Dest,TWF_Dest,TXK_Dest,TYR_Dest,TYS_Dest,VCT_Dest,VIS_Dest,VLD_Dest,VPS_Dest,WRG_Dest,WYS_Dest,XNA_Dest,YAK_Dest,YUM_Dest,day,evening,morning,night
0,-5.000000e-01,-0.866025,1,21,0,0,6.595781,19,34,361,130321,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,-5.000000e-01,0.866025,0,20,0,0,6.726233,15,48,225,50625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,-1.836970e-16,-1.000000,0,2,0,0,6.030685,14,22,196,38416,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,8.660254e-01,-0.500000,1,25,0,0,6.770789,10,15,100,10000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5.000000e-01,-0.866025,1,7,0,0,6.047372,18,28,324,104976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [16]:
cross_val_score(xgb_first, X=X_train_new.values, y=y_train, cv=skf, scoring="roc_auc", verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.3min finished


array([0.71732365, 0.72188616, 0.72212846])

In [17]:
cross_val_score(xgb, X=X_train_new.values, y=y_train, cv=skf, scoring="roc_auc", verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 29.5min finished


array([0.73809499, 0.74414352, 0.74085482])

In [23]:
X_train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 615 entries, Seasons_cos to night
dtypes: Sparse[int64, 0](578), float64(3), int64(8), uint8(26)
memory usage: 13.2 MB


In [24]:
xgb.fit(X_train_new.values, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.1,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=2, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=1.4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsamle=0.9, subsample=1, verbosity=1)

In [25]:
mf_test = MakeFeatures(is_weekend=1, 
                  seasons=1, 
                  day_of_month=1, 
                  dep_capital=1, 
                  arr_capital=1, 
                  route=1, 
                  log_dist=1, 
                  uc=1, 
                  time = 1,
                  new_dep_time = 1,
                  hour_imp = 1)
mf_test.fit(test)

X_test = mf_test.fit_transform(test)
X_test.head()

X_test.drop(['9E'], axis=1, inplace=True)

X_test['DH'] = 0
X_test['HP'] = 0
X_test['TZ'] = 0

X_test.Day = X_test.Day.map(int)
neworder = X_train_new.columns
X_test_new = X_test.reindex(columns=neworder)

Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Hour_sq-s feature complete!
Route complete!
Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Hour_sq-s feature complete!
Route complete!


In [0]:
predict_kaggle = xgb.predict_proba(X_test_new.values)[:, 1]

In [28]:
predict_kaggle

array([0.03998995, 0.05900993, 0.03185375, ..., 0.21175021, 0.11470821,
       0.0861673 ], dtype=float32)

In [0]:
make_predict(predict_kaggle,'xgb_all_data')